# UFMG / Latin

In [1]:
#DATAPATH = '../data3-mag2'
#MAGPATH  = '../../mag2'
#CPPPATH  = '../cpp'

DATAPATH = '../data3'
MAGPATH  = '../../mag3'
CPPPATH  = '../cpp'

PATHS = dict(CPPPATH=CPPPATH,MAGPATH=MAGPATH,DATAPATH=DATAPATH)
print PATHS

import sys
sys.path.append('../ipynb-mag2/') # python lib path
import cuplib

import pandas as pd
import numpy as np
import commands
%pylab inline

{'CPPPATH': '../cpp', 'DATAPATH': '../data3', 'MAGPATH': '../../mag3'}
Populating the interactive namespace from numpy and matplotlib


## Data

### citcount_p (NB02)

In [2]:
# cpp/citcount_p.cpp
# g++ ../cpp/citcount_p.cpp -o ../cpp/citcount_p -O3 
# time ../cpp/citcount_p ../../mag3/PaperReferences.txt ../data3/citcount_p.tsv

cmd = 'g++ %(CPPPATH)s/citcount_p.cpp -o %(CPPPATH)s/citcount_p -O3 && \
time %(CPPPATH)s/citcount_p %(MAGPATH)s/PaperReferences.txt %(DATAPATH)s/citcount_p.tsv' % PATHS
print cmd

print commands.getoutput('%s > %s/run.log' % (cmd,PATHS['DATAPATH']))

# 7m24.078s

g++ ../cpp/citcount_p.cpp -o ../cpp/citcount_p -O3 && time ../cpp/citcount_p ../../mag3/PaperReferences.txt ../data3/citcount_p.tsv
411.85user 59.82system 7:55.33elapsed 99%CPU (0avgtext+0avgdata 8405924maxresident)k
35341656inputs+1040280outputs (0major+123119minor)pagefaults 0swaps


In [3]:
%%time 
dfpcits = pd.read_csv(DATAPATH+'/citcount_p.tsv',
                      sep='\t',header=None,names='Pid Pcits'.split(),dtype=np.int32)
dfpcits.set_index('Pid',inplace=True)

CPU times: user 7.38 s, sys: 444 ms, total: 7.82 s
Wall time: 7.82 s


### ACR (NB12)

In [4]:
Xy = dfpcits

In [5]:
%%time
# 7min
dfp = cuplib.read_papers(magpath=MAGPATH,usecols='Pkey Pyear'.split())

CPU times: user 4min 3s, sys: 25.3 s, total: 4min 28s
Wall time: 5min 19s


In [6]:
%%time
Xy = Xy.join(dfp,how='outer')

CPU times: user 22.7 s, sys: 4.74 s, total: 27.5 s
Wall time: 27.6 s


In [7]:
%%time
sbmsource = 'final'
field = 'ACR' # S02_PcitsYear
Xy.Pyear.loc[Xy.Pyear > 2015] = 2015
Xy[field] = Xy.Pcits / ( 2015 + 1 - Xy.Pyear )

CPU times: user 6.86 s, sys: 2.38 s, total: 9.24 s
Wall time: 5.26 s


In [8]:
%%time
cuplib.write_submission(Xy[field],DATAPATH+'/%s/%s/'%(sbmsource,field),hexids=False)

CPU times: user 1min 43s, sys: 6.58 s, total: 1min 50s
Wall time: 1min 50s


'../data3/final/ACR/results.tsv'

In [9]:
#Xy['Pyear'].max()

### Graph

In [10]:
# hex2int
cmd = 'g++ %(CPPPATH)s/randomwalk_build_graph/hex2int.cpp -o %(CPPPATH)s/randomwalk_build_graph/hex2int -O3 && \
time %(CPPPATH)s/randomwalk_build_graph/hex2int %(MAGPATH)s/PaperReferences.txt %(DATAPATH)s/PaperReferences.tsv' \
% PATHS
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

# 10m58.763s

g++ ../cpp/randomwalk_build_graph/hex2int.cpp -o ../cpp/randomwalk_build_graph/hex2int -O3 && time ../cpp/randomwalk_build_graph/hex2int ../../mag3/PaperReferences.txt ../data3/PaperReferences.tsv
431.67user 25.13system 13:36.73elapsed 55%CPU (0avgtext+0avgdata 1508maxresident)k
34576752inputs+39949312outputs (4major+722minor)pagefaults 0swaps


In [11]:
# sd2nodes
cmd = 'g++ %(CPPPATH)s/randomwalk_build_graph/sd2nodes.cpp \
-o %(CPPPATH)s/randomwalk_build_graph/sd2nodes -O3 -std=c++11 && \
time %(CPPPATH)s/randomwalk_build_graph/sd2nodes \
%(DATAPATH)s/PaperReferences.tsv %(DATAPATH)s/PaperReferences' % PATHS
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

# 5m13.549s

g++ ../cpp/randomwalk_build_graph/sd2nodes.cpp -o ../cpp/randomwalk_build_graph/sd2nodes -O3 -std=c++11 && time ../cpp/randomwalk_build_graph/sd2nodes ../data3/PaperReferences.tsv ../data3/PaperReferences
300.99user 13.72system 5:32.37elapsed 94%CPU (0avgtext+0avgdata 1627544maxresident)k
36544280inputs+1035208outputs (3major+5274685minor)pagefaults 0swaps


In [12]:
# sd2inlinks
cmd = 'g++ %(CPPPATH)s/randomwalk_build_graph/sd2inlinks_int.cpp \
-o %(CPPPATH)s/randomwalk_build_graph/sd2inlinks_int -O3 -std=c++11 && \
time %(CPPPATH)s/randomwalk_build_graph/sd2inlinks_int \
%(DATAPATH)s/PaperReferences.tsv %(DATAPATH)s/PaperReferences.nodes %(DATAPATH)s/PaperReferences' % PATHS
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

# 27m30.578s

g++ ../cpp/randomwalk_build_graph/sd2inlinks_int.cpp -o ../cpp/randomwalk_build_graph/sd2inlinks_int -O3 -std=c++11 && time ../cpp/randomwalk_build_graph/sd2inlinks_int ../data3/PaperReferences.tsv ../data3/PaperReferences.nodes ../data3/PaperReferences
1378.42user 102.11system 26:12.11elapsed 94%CPU (0avgtext+0avgdata 32679204maxresident)k
35049592inputs+17275928outputs (5major+131137043minor)pagefaults 0swaps


## Running RCR

In [13]:
%%time 

# (sabir@host16) ../data2/sbm12/S02_PcitsYear/results.tsv

# curinga@host17:~$ cd /mnt/hd0/alberto/
# g++ acr.cpp -o acr -O3 -std=c++11
# time ./acr graph.nodes graph.inlinks results.tsv output-unique-bros.tsv > log-unique-bros.txt

# sabir@host16:~/wsdmcup/data2/model02mc$ wc -l graph.nodes

import os
try: os.makedirs('%(DATAPATH)s/final/sRCR11/' % PATHS) # TODO
except: pass

cmd = 'g++ %(CPPPATH)s/acr3.cpp -o %(CPPPATH)s/acr3 -O3 -std=c++11 && \
time %(CPPPATH)s/acr3 \
%(DATAPATH)s/PaperReferences.nodes %(DATAPATH)s/PaperReferences.inlinks %(DATAPATH)s/final/ACR/results.tsv \
%(DATAPATH)s/final/sRCR11.tsv' % PATHS # TODO
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

# 

g++ ../cpp/acr3.cpp -o ../cpp/acr3 -O3 -std=c++11 && time ../cpp/acr3 ../data3/PaperReferences.nodes ../data3/PaperReferences.inlinks ../data3/final/ACR/results.tsv ../data3/final/sRCR11.tsv
2655.84user 108.41system 46:14.13elapsed 99%CPU (0avgtext+0avgdata 27207204maxresident)k
2735176inputs+2045896outputs (5major+204373935minor)pagefaults 0swaps
CPU times: user 27.4 ms, sys: 195 ms, total: 223 ms
Wall time: 46min 16s


In [14]:
%%time 

df = pd.read_csv(DATAPATH+'/final/sRCR11.tsv', # TODO
                      sep='\t',header=None,names='Pid sRCR'.split(),index_col=0,nrows=None)

df['sRCR'][df['sRCR'] == 0.0] = 1.0

cuplib.write_submission(df['sRCR'],DATAPATH+'/final/sRCR11/',hexids=True) # TODO

CPU times: user 2min 8s, sys: 7.53 s, total: 2min 15s
Wall time: 2min 17s


In [15]:
#df.sRCR.tolist()